# House Prices: Advanced Regression Techniques
## 予測モデリングの作成

<H2>1. 必要なライブラリを読み込む</H2>

In [319]:
% matplotlib inline

import copy
import os
import sys
import datetime

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

import pyodbc # データベースに接続するため

matplotlib.style.use('ggplot')

from __future__ import print_function

<H2>2. 定数定義</H2>

In [320]:
CONST_RANDOM_STATE = 1234
CONST_TEST_SIZE = 0.3
CONST_N_JOBS = -1
CONST_CUTOFF_R = 0.5
CONST_GRID_SEARCH_PARAMS = {'n_estimators': [200, 225, 250], 'max_depth': [10, 15, 20]}
#CONST_GRID_SEARCH_PARAMS = {'n_estimators': [225], 'max_depth': [20]}
CONST_FIRST_RMSLE = 1

<H2>3. グローバル変数定義</H2>

In [321]:
gRfParams = {'n_estimators' : 225, 'max_depth' : 15}
gDataSet = None
gDataSetTest = None
gColNames = None
gMissingCheck = None
gTargetCol = None
gExcludeCols = None
gFeatureCols = None
gRfr = None
gRmsle = CONST_FIRST_RMSLE
gNewXTrain1 = None
gNewXTrain2 = None
gNewXTrain = None
gNewXTest = None
gNewRealXTest = None

<H2>4. データの読み込み</H2>

In [322]:
def get_DataSet(sql) :
    # CSV ファイルからデータを読み込む
    # gDataSet = pd.read_csv('dataset/train.csv')
    
    # SQL Server からデータを読み込む
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=.\SQLEXPRESS;DATABASE=HousePrices;UID=sa;PWD=pass@word1')

    # Query database and load the returned results in pandas data frame
    dataSet = pd.read_sql(sql, conn)
    #dataSet = pd.read_sql('''select * from train''', conn)
    
    return dataSet

In [323]:
gDataSet = get_DataSet('select * from train')

In [324]:
gDataSetTest = get_DataSet('select * from test')

<h2>5. 特徴量の作成</h2>

<h3>ダミー変数を作る</h3>
<p> 一般的にはカテゴリ変数を機械学習のモデルに投入する際、0と1のダミー変数に変換する。</p>

In [325]:
# view のデータの種類を確認
# dataset['view'].unique()
# dataset = pd.get_dummies(data=dataset, columns=['view'])

<h3>カラム間の演算</h3>
<p>"LotFrontage/(LotArea/LotFrontage)"を新たな列として加える。土地の形を長方形と考えた場合において、間口と奥行きの比率を表す。</p>

In [326]:
# 学習データ
gDataSet['LotRatio'] = gDataSet['LotFrontage_Avg']/(gDataSet['LotArea']/gDataSet['LotFrontage_Avg'])
gDataSet.ix[0:9][['Id', 'LotArea', 'LotFrontage_Avg', 'LotRatio']]
# 検証データ
gDataSetTest['LotRatio'] = gDataSetTest['LotFrontage_Avg']/(gDataSetTest['LotArea']/gDataSetTest['LotFrontage_Avg'])
gDataSetTest.ix[0:9][['Id', 'LotArea', 'LotFrontage_Avg', 'LotRatio']]

C:\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
C:\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Id,LotArea,LotFrontage_Avg,LotRatio
0,1461,11622.0,80.0,0.550680
1,1462,14267.0,81.0,0.459872
2,1463,13830.0,74.0,0.395951
3,1464,9978.0,78.0,0.609741
4,1465,5005.0,43.0,0.369431
5,1466,10000.0,75.0,0.562500
6,1467,7980.0,65.0,0.529449
7,1468,8402.0,63.0,0.472388
8,1469,10176.0,85.0,0.710004
9,1470,8400.0,70.0,0.583333


<h2>6. ランダムフォレスト用いた予測モデルの構築</h2>

<p>ランダムフォレストを用いた機械学習アルゴリズムを試します</p>
<p>モデル評価はRMSLEとします</p>

In [327]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_squared_error

<h3>ターゲット変数と特徴量を指定してsklearnに渡せるように準備する</h3>

In [328]:
gTargetCol = 'SalePrice'
gExcludeCols = ['SalePrice',
                'LotFrontage',
                'GarageYrBlt',
                'Alley',
                'BldgType',
                'BsmtCond',
                'BsmtExposure',
                'BsmtFinType1',
                'BsmtFinType2',
                'BsmtQual',
                'CentralAir',
                'Condition1',
                'Condition2',
                'Electrical',
                'ExterCond',
                'Exterior1st',
                'Exterior2nd',
                'ExterQual',
                'Fence',
                'FireplaceQu',
                'Foundation',
                'Functional',
                'GarageCond',
                'GarageFinish',
                'GarageQual',
                'GarageType',
                'Heating',
                'HeatingQC',
                'HouseStyle',
                'KitchenQual',
                'LandContour',
                'LandSlope',
                'LotConfig',
                'LotShape',
                'MasVnrType',
                'MiscFeature',
                'MSSubClass',
                'MSZoning',
                'Neighborhood',
                'OverallCond',
                'OverallQual',
                'PavedDrive',
                'PoolQC',
                'RoofMatl',
                'RoofStyle',
                'SaleCondition',
                'SaleType',
                'Street',
                'Utilities']

gFeatureCols = [col for col in gDataSet.columns if col not in gExcludeCols]

<H3>特徴量を割り算で作成</H3>

In [329]:
def get_NewAddedCols() :
    new_added_col = []
    for i in range(0, len(gFeatureCols)-1):
        for j in range(i+1, len(gFeatureCols)):
            first_col_name = gFeatureCols[i]
            second_col_name = gFeatureCols[j]
            r = spearmanr(gDataSet[first_col_name], gDataSet[second_col_name]).correlation
            if abs(r) > CONST_CUTOFF_R:
                new_colname = first_col_name + "_div_" + second_col_name
                gDataSet[new_colname] = gDataSet[first_col_name] / (gDataSet[second_col_name] + 0.01)
                gDataSetTest[new_colname] = gDataSetTest[first_col_name] / (gDataSetTest[second_col_name] + 0.01)
                new_added_col.append(new_colname)
    return new_added_col

In [330]:
gFeatureCols = gFeatureCols + get_NewAddedCols()
gFeatureCols

['Id',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'Alley_INT',
 'BldgType_INT',
 'BsmtCond_INT',
 'BsmtExposure_INT',
 'BsmtFinType1_INT',
 'BsmtFinType2_INT',
 'BsmtQual_INT',
 'CentralAir_INT',
 'Condition1_INT',
 'Condition2_INT',
 'Electrical_INT',
 'ExterCond_INT',
 'Exterior1st_INT',
 'Exterior2nd_INT',
 'ExterQual_INT',
 'Fence_INT',
 'FireplaceQu_INT',
 'Foundation_INT',
 'Functional_INT',
 'GarageCond_INT',
 'GarageFinish_INT',
 'GarageQual_INT',
 'GarageType_INT',
 'Heating_INT',
 'HeatingQC_INT',
 'HouseStyle_INT',
 'KitchenQual_INT',
 'LandContour_INT',
 'LandSlope_I

<H3>説明変数（特徴量）</H3>

In [331]:
gDataSet[gFeatureCols].head()

,Id,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,...,ExterQual_INT_div_HeatingQC_INT,ExterQual_INT_div_KitchenQual_INT,ExterQual_INT_div_OverallQual_INT,ExterQual_INT_div_GarageYrBlt_Oldest,Foundation_INT_div_GarageYrBlt_Oldest,GarageCond_INT_div_GarageQual_INT,KitchenQual_INT_div_OverallQual_INT,LandContour_INT_div_LandSlope_INT,OverallQual_INT_div_GarageYrBlt_Oldest,LotFrontage_Avg_div_LotRatio
0,1,8450.0,2003,2003,196,706,0,150,856,856,...,2.970297,0.996678,0.374532,0.001498,0.001498,0.998336,0.374532,3.960396,0.003994,127.450980
1,2,9600.0,1976,1976,0,978,0,284,1262,1262,...,3.960396,0.997506,0.570613,0.002024,0.001012,0.998336,0.570613,3.960396,0.003542,118.226601
2,3,11250.0,2001,2002,162,486,0,434,920,920,...,2.970297,0.996678,0.374532,0.001499,0.001499,0.998336,0.374532,3.960396,0.003998,161.511665
3,4,9550.0,1915,1970,0,216,0,540,756,961,...,1.328904,1.328904,0.499376,0.002002,0.000500,0.998336,0.374532,3.960396,0.004004,155.053443
4,5,14260.0,2000,2000,350,655,0,490,1145,1145,...,2.970297,0.996678,0.332963,0.001500,0.001500,0.998336,0.332963,3.960396,0.004500,166.399022


<H3>被説明変数（ターゲット変数）</H3>

In [332]:
gDataSet[gTargetCol].head()

0    208500.0
1    181500.0
2    223500.0
3    140000.0
4    250000.0
Name: SalePrice, dtype: float64

<H3>機械学習アルゴリズムに投入するための NumPy 配列</H3>

In [333]:
#gIncludeCols = ["Id","LotArea","YearBuilt","MasVnrArea","BsmtFinSF1","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","TotRmsAbvGrd","GarageArea","OpenPorchSF","CentralAir_INT","Neighborhood_INT","OverallCond_INT","OverallQual_INT","LotFrontage_Avg","LotRatio","LotArea_div_LotFrontage_Avg","YearBuilt_div_FullBath","YearBuilt_div_BsmtQual_INT","YearBuilt_div_ExterQual_INT","YearBuilt_div_GarageFinish_INT","YearBuilt_div_OverallQual_INT","YearRemodAdd_div_ExterQual_INT","YearRemodAdd_div_KitchenQual_INT","YearRemodAdd_div_OverallQual_INT","BsmtFinSF1_div_BsmtUnfSF","BsmtFinSF1_div_BsmtFullBath","BsmtFinSF1_div_BsmtFinType1_INT","GrLivArea_div_FullBath","GrLivArea_div_BedroomAbvGr","GrLivArea_div_TotRmsAbvGrd","GrLivArea_div_GarageCars","GrLivArea_div_OverallQual_INT","FullBath_div_BsmtQual_INT","FullBath_div_GarageYrBlt_Oldest","GarageCars_div_GarageArea","GarageCars_div_BsmtQual_INT","GarageCars_div_ExterQual_INT","GarageCars_div_OverallQual_INT","GarageCars_div_GarageYrBlt_Oldest","BsmtQual_INT_div_OverallQual_INT","ExterQual_INT_div_OverallQual_INT","ExterQual_INT_div_GarageYrBlt_Oldest","KitchenQual_INT_div_OverallQual_INT","OverallQual_INT_div_GarageYrBlt_Oldest"]
#51
#gIncludeCols = ["Id","LotArea","YearBuilt","MasVnrArea","BsmtFinSF1","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","GarageArea","OpenPorchSF","CentralAir_INT","MSZoning_INT","Neighborhood_INT","OverallCond_INT","OverallQual_INT","LotFrontage_Avg","LotRatio","LotArea_div_LotFrontage_Avg","YearBuilt_div_FullBath","YearBuilt_div_BsmtQual_INT","YearBuilt_div_ExterQual_INT","YearBuilt_div_GarageFinish_INT","YearBuilt_div_OverallQual_INT","YearRemodAdd_div_BsmtQual_INT","YearRemodAdd_div_ExterQual_INT","YearRemodAdd_div_KitchenQual_INT","YearRemodAdd_div_OverallQual_INT","BsmtFinSF1_div_BsmtUnfSF","BsmtFinSF1_div_BsmtFullBath","BsmtFinSF1_div_BsmtFinType1_INT","TotalBsmtSF_div_1stFlrSF","GrLivArea_div_FullBath","GrLivArea_div_BedroomAbvGr","GrLivArea_div_TotRmsAbvGrd","GrLivArea_div_GarageCars","GrLivArea_div_OverallQual_INT","FullBath_div_BsmtQual_INT","FullBath_div_GarageYrBlt_Oldest","GarageCars_div_GarageArea","GarageCars_div_BsmtQual_INT","GarageCars_div_ExterQual_INT","GarageCars_div_OverallQual_INT","GarageCars_div_GarageYrBlt_Oldest","BsmtQual_INT_div_OverallQual_INT","ExterQual_INT_div_OverallQual_INT","ExterQual_INT_div_GarageYrBlt_Oldest","KitchenQual_INT_div_OverallQual_INT","OverallQual_INT_div_GarageYrBlt_Oldest"]
#53
gIncludeCols = ["Id","LotArea","YearBuilt","MasVnrArea","BsmtFinSF1","BsmtUnfSF","TotalBsmtSF","1stFlrSF","2ndFlrSF","GrLivArea","FullBath","TotRmsAbvGrd","GarageArea","WoodDeckSF","OpenPorchSF","CentralAir_INT","MSZoning_INT","Neighborhood_INT","OverallCond_INT","OverallQual_INT","LotFrontage_Avg","LotRatio","LotArea_div_LotFrontage_Avg","YearBuilt_div_FullBath","YearBuilt_div_BsmtQual_INT","YearBuilt_div_ExterQual_INT","YearBuilt_div_GarageFinish_INT","YearBuilt_div_OverallQual_INT","YearRemodAdd_div_BsmtQual_INT","YearRemodAdd_div_ExterQual_INT","YearRemodAdd_div_KitchenQual_INT","YearRemodAdd_div_OverallQual_INT","BsmtFinSF1_div_BsmtUnfSF","BsmtFinSF1_div_BsmtFullBath","BsmtFinSF1_div_BsmtFinType1_INT","TotalBsmtSF_div_1stFlrSF","GrLivArea_div_FullBath","GrLivArea_div_BedroomAbvGr","GrLivArea_div_TotRmsAbvGrd","GrLivArea_div_GarageCars","GrLivArea_div_OverallQual_INT","FullBath_div_BsmtQual_INT","FullBath_div_GarageYrBlt_Oldest","GarageCars_div_GarageArea","GarageCars_div_BsmtQual_INT","GarageCars_div_ExterQual_INT","GarageCars_div_OverallQual_INT","GarageCars_div_GarageYrBlt_Oldest","BsmtQual_INT_div_OverallQual_INT","ExterQual_INT_div_OverallQual_INT","ExterQual_INT_div_GarageYrBlt_Oldest","KitchenQual_INT_div_OverallQual_INT","OverallQual_INT_div_GarageYrBlt_Oldest"]
#14
#gIncludeCols = ["LotArea","BsmtFinSF1","TotalBsmtSF","1stFlrSF","GrLivArea","YearBuilt_div_FullBath","YearBuilt_div_ExterQual_INT","YearBuilt_div_OverallQual_INT","YearRemodAdd_div_OverallQual_INT","FullBath_div_BsmtQual_INT","GarageCars_div_BsmtQual_INT","GarageCars_div_ExterQual_INT","ExterQual_INT_div_OverallQual_INT","OverallQual_INT_div_GarageYrBlt_Oldest"]
gFeatureCols = gIncludeCols
#gFeatureCols = [col for col in gDataSet.columns if col in gIncludeCols]

# DEBUG
print(len(gFeatureCols))

#被説明変数（ターゲット変数）のベクトル
gY = np.array(gDataSet[gTargetCol])
#NumPyの配列に直した特徴量の行列
gX = np.array(gDataSet[gFeatureCols])
gRealXTest = np.array(gDataSetTest[gFeatureCols])

53


In [334]:
# 学習データを70%(X_train, y_train)、テストデータを30%(X_test, y_test)に分割
gX_train, gX_test, gY_train, gY_test = train_test_split(gX, gY, test_size=CONST_TEST_SIZE, random_state=CONST_RANDOM_STATE)

<H3>ランダムフォレストの実行</H3>

<h5>ランダムフォレストの適合</h4>

In [335]:
def GetRmsle(real, predicted):
    sum=0.0
    for x in range(len(predicted)):
        p = np.log(predicted[x]+1)
        r = np.log(real[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(predicted))**0.5

<h2>7. テストデータへ適用して精度を確認する</h2>

<h3>モデルの学習</h3>

In [336]:
#モデルの作成
model = RandomForestRegressor(random_state=CONST_RANDOM_STATE, n_estimators=gRfParams['n_estimators'], max_depth=gRfParams['max_depth'], n_jobs=CONST_N_JOBS)
model.fit(gX_train, gY_train)
y_pred = model.predict(gX_test)
rmsle = GetRmsle(gY_test, y_pred)
print(rmsle)

### 4. パラメーターチューニング ###
gscv = GridSearchCV(model, param_grid=CONST_GRID_SEARCH_PARAMS, scoring='neg_mean_squared_log_error')
gscv.fit(gX_train, gY_train)
gRfParams['n_estimators'] = gscv.best_params_['n_estimators']
gRfParams['max_depth'] = gscv.best_params_['max_depth']
model = RandomForestRegressor(random_state=CONST_RANDOM_STATE, n_estimators=gRfParams['n_estimators'], max_depth=gRfParams['max_depth'], n_jobs=CONST_N_JOBS)
model.fit(gX_train, gY_train)
y_pred = model.predict(gX_test)
rmsle = GetRmsle(gY_test, y_pred)
print(rmsle)
print(gRfParams)

# 予測値の導出
y_pred_on_test = model.predict(gRealXTest)
# 予測値の出力
f = open('D:\\DataMix\\08.170909\\House Prices Advanced Regression Techniques\\dataset\\y_pred_on_test.txt', 'w')
for x in y_pred_on_test:
    f.write(str(x) + "\n")
f.close()
# DEBUG
print('y_pred_on_test')
print(y_pred_on_test)

0.148332608153
0.148332608153
{'n_estimators': 225, 'max_depth': 15}
y_pred_on_test
[ 125086.27982098  161205.24126984  180683.79528965 ...,  154484.58259259
  107830.9527562   229577.60992593]
